In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
import numpy as np

In [3]:
inputs = Input(shape=(20,))
out = Dense(10)(inputs)
model = Model(inputs, out)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
Total params: 210
Trainable params: 210
Non-trainable params: 0
_________________________________________________________________


In [5]:
X = np.random.rand(1,20)
X

array([[0.91709806, 0.82830839, 0.93914066, 0.57037095, 0.04271652,
        0.35534695, 0.29179199, 0.80227101, 0.65690956, 0.59359125,
        0.57083799, 0.64906287, 0.08615951, 0.20494363, 0.98687436,
        0.70022373, 0.8282763 , 0.38845018, 0.1025627 , 0.46584396]])

In [6]:
y = model.predict(X)
y

1/1 [==============================] - 0s 31ms/step


2023-10-23 20:08:46.369862: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[-0.9478299 , -0.2901961 ,  1.2173429 ,  0.9856129 ,  0.44817972,
         0.8500049 ,  1.2243729 ,  1.1230452 , -0.8100219 ,  0.65824366]],
      dtype=float32)

In [7]:
X_in = [int(x*1e36) for x in X[0]]
weights = [[None for _ in range(10)] for _ in range(20)]
for i in range(20):
    for j in range(10):
        weights[i][j] = int(model.get_weights()[0][i][j]*1e36)
bias = [int(b*1e72) for b in model.get_weights()[1]]

In [8]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder
        

In [9]:
X_in, weights, bias, out, remainder = DenseInt(20, 10, 10**36, X_in, weights, bias)
out, remainder

(['21888242871839275222246405745257275088547416570344934017690618098050994599407',
  '21888242871839275222246405745257275088548074204322728883645171911568575253269',
  '1217342773824182708964164664298235747',
  '985612918881516198369842470059289901',
  '448179765935532159653151439285979820',
  '850005025728080684762697042708611430',
  '1224373017260726380443809993782617513',
  '1123045202502242457956309189904120564',
  '21888242871839275222246405745257275088547554378545433689485043918282349046950',
  '658243650814707862463466520227993190'],
 ['51492294878586576180273547728388096',
  '777789140836427041572477556551057408',
  '890475291130181473765283908913463296',
  '963682024802736049277914862344732672',
  '180838407593997560156976766263492608',
  '458215330546393498330258578539020288',
  '738904904497555276279419263936102400',
  '770453107465054933435045502301241344',
  '616225701137188004915996184419500032',
  '106851476518473341575934978717384704'])

In [10]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "remainder": remainder
}

In [11]:
import json

In [12]:
with open("dense_input.json", "w") as f:
    json.dump(in_json, f)